This Notebook launches a study comparing the different variants of numerical fluxes for a Mach 4 supersonic flow over a blunt body presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024) 

In [9]:
#r "BoSSSpad.dll"
#r "XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 217
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 104
   at Submission#10.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

init workflow


In [10]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_FluxStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_BowShock_FluxStudy'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESF_BowShock_FluxStudy'.


{ Session Count = 4; Grid Count = 4; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_FluxStudy }

Get Default Queue

In [11]:
BoSSSshell.ExecutionQueues

index,type,value
0,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client Default @DC3, @\\dc3\userspace\sebastian\cluster\binaries"
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client Default @DC3, @\\dc3\userspace\sebastian\cluster\binaries"
2,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,MiniBatchProcessor client @C:\Users\sebastian\AppData\Local\BoSSS-LocalJobs


default database path

In [12]:
BoSSSshell.WorkflowMgm.DefaultDatabase.Path

\\dc3\userspace\sebastian\cluster\XESF_BowShock_FluxStudy

list of available queues

In [13]:
BoSSSshell.GetDefaultQueue()

RuntimeLocation,win\amd64
AdditionalEnvironmentVars,"[ [OMP_PROC_BIND, spread] ]"
DeploymentBaseDirectory,\\dc3\userspace\sebastian\cluster\binaries
DeployRuntime,True
Name,Default
DotnetRuntime,dotnet
Username,FDY\sebastian
NumOfAdditionalServiceCores,0
NumOfAdditionalServiceCoresMPISerial,0
NumOfServiceCoresPerMPIprocess,0
ServerName,DC3


Here we prescribe all combinations we want to study

In [14]:
using XESF.Fluxes;
using ApplicationWithIDT;

int[] DGdegrees = new int[] {2};
int[] Grid_Res = new int[] {2};
double[] Aggs = new double[] {0.4};
var iFluxes = new int[] {0,1,2,3}; //{ ConvectiveInterfaceFluxes.GodunovInterface,ConvectiveInterfaceFluxes.RoeInterface, ConvectiveInterfaceFluxes.CentralFluxInterface, ConvectiveInterfaceFluxes.OptimizedHLLCInterface };
var wallFluxes=new int[] {0}; //{ ConvectiveInterfaceFluxes.OptimizedHLLCWall, ConvectiveInterfaceFluxes.RoeWall};
var bulkFluxes=new int[] {0}; // { ConvectiveBulkFluxes.OptimizedHLLC, ConvectiveBulkFluxes.Roe, ConvectiveBulkFluxes.CentralFlux, ConvectiveBulkFluxes.Godunov};
var iProbs= new int[] {0}; //{ OptProblemType.FullEnRes, OptProblemType.EnResOnlyNearBand, OptProblemType.RankineHugoniot};
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;


get data we need to initialize the initial guess for the shock LevelSet

In [15]:
var bowShockPoints =new Tuple<byte[],string>( System.IO.File.ReadAllBytes("BowShockPoints.txt"),"BowShockPoints.txt");
var LevelSetDB =new Tuple<byte[],string>(System.IO.File.ReadAllBytes("bosss_db_levelSets.zip"),"bosss_db_levelSets.zip");

run the study

In [16]:
foreach(int iProb in iProbs) {
foreach(int iflux in iFluxes) {
    foreach(int wflux in wallFluxes) {
        foreach(int bflux in bulkFluxes) {
foreach(double agg in Aggs) {
    foreach(int p in DGdegrees) {
        foreach(int n in Grid_Res) {
                var ctrl_temp=XESF.XESFHardCodedControl.XDGBS_Cluster(DegE: p ,numX: 5*n ,numY: 16*n ,agg: agg ,plotInterval: -1 ,iProb: iProb ,iflux: iflux,bulkflux:bflux,wallflux:wflux,aRI:true,dbPath:  dbPath  );
                Job j = new Job(ctrl_temp.SessionName,new XESF.XESFMain().GetType()); // job name has to be equal to sessionname! for SessionInfoJobCorrelation to work!
                
                j.RetryCount = 2;
                j.MySetCommandLineArguments("--control", $"cs:XESF.XESFHardCodedControl.XDGBS_Cluster(DegE:{p},numX:{5*n},numY:{16*n},agg:{agg},plotInterval:{-1},iProb:{iProb},iflux:{iflux},bulkflux:{bflux},wallflux:{wflux},aRI:true,dbPath:@\"{dbPath}\")");
                j.AdditionalDeploymentFiles.Add(bowShockPoints);
                j.AdditionalDeploymentFiles.Add(LevelSetDB);
                j.NumberOfThreads=1;
                Console.WriteLine("!!!!!!!!!!!!!!!! WARNING: OPENMP Paralelization turned off !!!!!!!!!!!!");
                j.Activate(BoSSSshell.ExecutionQueues[0]); 
        };                                                                                                                                                          
    };
    }
        }
        }
};
}


Error: System.ArgumentException: Job with name 'FS-XDGBS-p2-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0' is already defined in the workflow management.
   at BoSSS.Application.BoSSSpad.Job..ctor(String name, Type solver) in C:\experimental\public\src\L4-application\BoSSSpad\Job.cs:line 66
   at Submission#17.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)